In [1]:
import numpy as np
import keras
from keras import backend as K
from keras.layers.core import Dense
from keras.optimizers import Adam,RMSprop
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint,EarlyStopping
from keras.applications import imagenet_utils,MobileNet
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
MobileNet=MobileNet(weights='imagenet',include_top=False,input_shape=(224,224,3))

17227776/17225924 [==============================] - 0s 0us/step


In [3]:
for layer in MobileNet.layers:
    layer.trainable=False

In [4]:
def fullyConn(bottom_Model):
    top_model=bottom_Model.output
    top_model=keras.layers.GlobalAveragePooling2D()(top_model)
    top_model=keras.layers.Dense(1024,activation='relu')(top_model)
    top_model=keras.layers.Dense(1024,activation='relu')(top_model)
    top_model=keras.layers.Dense(512,activation='relu')(top_model)
    top_model=keras.layers.Dense(2,activation='softmax')(top_model)
    
    return top_model
    

In [5]:
new_mod=fullyConn(MobileNet)

model=Model(inputs=MobileNet.input,outputs=new_mod)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [11]:
train_path="/content/drive/My Drive/CovidDataset/Train"
valid_path='/content/drive/My Drive/CovidDataset/Val'

In [7]:
train_datagen=ImageDataGenerator(rescale=1./255)

In [8]:
validation_datagen=ImageDataGenerator(rescale=1./255)

In [9]:
batch_size=16

In [13]:
train_generator=train_datagen.flow_from_directory(train_path,target_size=(224,224),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')
validation_generator=validation_datagen.flow_from_directory(valid_path,
                                                           target_size=(224,224),
                                                           batch_size=batch_size,
                                                           class_mode='categorical')

Found 224 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [14]:
checkpoint=ModelCheckpoint("Corona.h5",monitor="val_loss",mode="min",save_best_only=True,verbose=1)
earlystop=EarlyStopping(monitor="val_loss",min_delta=0,patience=3,verbose=1,restore_best_weights=True)
callbacks=[checkpoint,earlystop]

In [15]:
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['accuracy'])

In [16]:
history=model.fit_generator(train_generator,steps_per_epoch=688/16,epochs=30,
                           callbacks=callbacks,validation_data=validation_generator,validation_steps=688/16)

Epoch 1/30
43/43 [==============================] - 198s 5s/step - loss: 2.6074 - accuracy: 0.8634 - val_loss: 0.1408 - val_accuracy: 0.9660

Epoch 00001: val_loss improved from inf to 0.14084, saving model to Corona.h5
Epoch 2/30
43/43 [==============================] - 108s 3s/step - loss: 0.1646 - accuracy: 0.9608 - val_loss: 0.3395 - val_accuracy: 0.9006

Epoch 00002: val_loss did not improve from 0.14084
Epoch 3/30
43/43 [==============================] - 106s 2s/step - loss: 0.1174 - accuracy: 0.9622 - val_loss: 1.4396 - val_accuracy: 0.8012

Epoch 00003: val_loss did not improve from 0.14084
Epoch 4/30
43/43 [==============================] - 107s 2s/step - loss: 0.1458 - accuracy: 0.9753 - val_loss: 0.5423 - val_accuracy: 0.5497

Epoch 00004: val_loss did not improve from 0.14084
Restoring model weights from the end of the best epoch
Epoch 00004: early stopping
